## Data Processing

In this notebook, we will create a DataProcessor class that will do the following 

1. Remove certain columns
2. Add some columns related to date time
3. Deal with missing values
4. One hot encoding

##### The DataProcessor class will follow fit and transform template so that it can be used in the machine learning pipelines

In [ ]:
import sys
import inspect
import warnings

In [ ]:
import numpy as np
import pandas as pd

In [23]:
# Add the scripts directory to the sys path
sys.path.append("../src/data")

from make_dataset import get_data

In [24]:
# suppres warnings
warnings.filterwarnings("ignore")

# Show all rows and columns in the display
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [26]:
print(inspect.getsource(get_data))

def get_data(data_string):
    """
    Read the train/test dataset and merge with properties data set and remove duplicate parcelid's in train
    
    Parameters:
    data_string -- "train" or "test" 
    
    Returns:
    X, y -- a tuple of dataframe X and Series y
    
    
    """         
    year = 2016 if data_string == "train" else 2017
        
    train = read_data("train_{0}".format(year))
    properties = read_data("properties_{0}".format(year))
    merged = pd.merge(train, properties, on="parcelid", how="left")
                      
    if data_string == "train":
        merged = remove_duplicate_parcels(merged)
                          
    y = merged["logerror"]                          
    merged = merged.drop(columns=["logerror"], axis=1) 
    
    id_col = ["parcelid"]
    cat_cols = ["airconditioningtypeid", "architecturalstyletypeid", "buildingclasstypeid", 
                "buildingqualitytypeid", "decktypeid", "fips", "fireplaceflag", 
                "hashottu

In [27]:
X_train, y_train = get_data(data_string="train")

In [28]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 12731
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   parcelid                      90150 non-null  object        
 1   transactiondate               90150 non-null  datetime64[ns]
 2   airconditioningtypeid         28748 non-null  object        
 3   architecturalstyletypeid      260 non-null    object        
 4   basementsqft                  43 non-null     float64       
 5   bathroomcnt                   90150 non-null  float64       
 6   bedroomcnt                    90150 non-null  float64       
 7   buildingclasstypeid           16 non-null     object        
 8   buildingqualitytypeid         57284 non-null  object        
 9   calculatedbathnbr             88974 non-null  float64       
 10  decktypeid                    658 non-null    object        
 11  finishedfloor1squarefeet    

In [29]:
len(y_train)

90150

### Drop certain columns

We will remove below columns before going ahead with the analysis

1. parcelid - this is not required for model training
2. propertyzoningdesc, rawcensustractandblock, regionidneighborhood, regionidzip, censustractandblock - these columns have lot of cardinality in these features. We will use regionidcity (177 unique groups) as we need one feature to distinguish the regions of the properties. 

In [30]:
class DataProcessor:
    def __init__(self, cols_to_remove=None):
        self.cols_to_remove = cols_to_remove

    def fit(self, X, y=None):
        """fit the process on the training data"""

        return self

    def transform(self, X, y=None):
        """transform the process on the train/test data """

        X_new = X.drop(columns=self.cols_to_remove, axis=1)

        return X_new

    def fit_transform(self, X, y=None):
        """fit and transform"""

        return self.fit(X).transform(X)

In [31]:
dp = DataProcessor(
    cols_to_remove=[
        "parcelid",
        "propertyzoningdesc",
        "rawcensustractandblock",
        "regionidneighborhood",
        "regionidzip",
        "censustractandblock",
    ]
)
X_train_new = dp.transform(X_train)

In [32]:
X_train_new.info()  # 6 columns are removed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 12731
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   transactiondate               90150 non-null  datetime64[ns]
 1   airconditioningtypeid         28748 non-null  object        
 2   architecturalstyletypeid      260 non-null    object        
 3   basementsqft                  43 non-null     float64       
 4   bathroomcnt                   90150 non-null  float64       
 5   bedroomcnt                    90150 non-null  float64       
 6   buildingclasstypeid           16 non-null     object        
 7   buildingqualitytypeid         57284 non-null  object        
 8   calculatedbathnbr             88974 non-null  float64       
 9   decktypeid                    658 non-null    object        
 10  finishedfloor1squarefeet      6850 non-null   float64       
 11  calculatedfinishedsquarefeet

## Add month and year column and remove transaction date column

We will need month and year columns as we will need to predict for 6 time points based on month and year 
(October 2016,  November 2016, December 2016, October 2017,  November 2017, December 2017)

In [33]:
class DataProcessor:
    def __init__(self, cols_to_remove=None, datecol=None):
        self.cols_to_remove = cols_to_remove
        self.datecol = datecol

    def fit(self, X, y=None):
        """fit the process on the training data"""

        return self

    def transform(self, X, y=None):
        """transform the process on the train/test data """

        X_new = X.drop(columns=self.cols_to_remove, axis=1)

        if self.datecol:
            X_new[self.datecol + "_month"] = pd.to_datetime(
                X_new[self.datecol]
            ).dt.month
            X_new[self.datecol + "_year"] = pd.to_datetime(X_new[self.datecol]).dt.year
            X_new = X_new.drop(columns=self.datecol, axis=1)

        return X_new

    def fit_transform(self, X, y=None):
        """fit and transform"""

        return self.fit(X).transform(X)

In [34]:
dp = DataProcessor(
    cols_to_remove=[
        "parcelid",
        "propertyzoningdesc",
        "rawcensustractandblock",
        "regionidneighborhood",
        "regionidzip",
        "censustractandblock",
    ],
    datecol="transactiondate",
)
X_train_new = dp.transform(X_train)

In [35]:
X_train_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 12731
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   airconditioningtypeid         28748 non-null  object 
 1   architecturalstyletypeid      260 non-null    object 
 2   basementsqft                  43 non-null     float64
 3   bathroomcnt                   90150 non-null  float64
 4   bedroomcnt                    90150 non-null  float64
 5   buildingclasstypeid           16 non-null     object 
 6   buildingqualitytypeid         57284 non-null  object 
 7   calculatedbathnbr             88974 non-null  float64
 8   decktypeid                    658 non-null    object 
 9   finishedfloor1squarefeet      6850 non-null   float64
 10  calculatedfinishedsquarefeet  89492 non-null  float64
 11  finishedsquarefeet12          85485 non-null  float64
 12  finishedsquarefeet13          33 non-null     float64
 13  f

## Deal with missing values

As we have seen in data exploratory notebook, there are lot of columns with missing values. We can not remove those records or columns as we will be losing some information that will help with predictions. Hence let's take a look at imputing these values.

We can impute with median/mean values but there are lot of records that are missing and hence median/mean may not make sense in that case. 
Lets look at imputing with negative values if there are no negative values in the columns..

In [36]:
X_train_new.describe()

,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,transactiondate_month,transactiondate_year
count,43.000000,90150.000000,90150.000000,88974.000000,6850.000000,89492.000000,85485.000000,33.000000,3555.000000,6850.000000,419.000000,9597.000000,88974.000000,29897.000000,29897.000000,9.015000e+04,9.015000e+04,8.001400e+04,17876.0,966.000000,90150.000000,11996.000000,58271.000000,2645.000000,95.000000,20540.000000,8.977200e+04,9.014900e+04,9.014900e+04,90144.000000,90150.000000,90150.0
mean,713.581395,2.279545,3.031936,2.309175,1347.720146,1773.096869,1745.415079,1404.545455,2380.880731,1355.299416,2293.069212,1.187871,2.241172,1.812055,345.539619,3.400547e+07,-1.181989e+08,2.912064e+04,1.0,519.688406,1.478580,1.008753,1.110244,310.204915,311.694737,1.440798,1.801032e+05,4.576379e+05,2.782879e+05,5983.070888,5.849917,2016.0
std,437.434198,1.004133,1.156114,0.976137,652.039469,928.136339,909.947071,110.108211,1068.859229,673.376506,1341.830257,0.484253,0.963106,0.608865,267.038335,2.649792e+05,3.606321e+05,1.217909e+05,0.0,155.116033,2.819802,0.100884,0.797389,216.738757,346.354850,0.544482,2.090769e+05,5.548532e+05,4.005040e+05,6838.506814,2.812363,0.0
min,100.000000,0.000000,0.000000,1.000000,44.000000,2.000000,2.000000,1056.000000,560.000000,44.000000,257.000000,1.000000,1.000000,0.000000,0.000000,3.333930e+07,-1.194479e+08,1.670000e+02,1.0,28.000000,0.000000,1.000000,1.000000,25.000000,18.000000,1.000000,1.000000e+02,2.200000e+01,2.200000e+01,49.080000,1.000000,2016.0
25%,407.500000,2.000000,2.000000,2.000000,938.000000,1184.000000,1172.000000,1392.000000,1648.000000,938.000000,1109.000000,1.000000,2.000000,2.000000,0.000000,3.381166e+07,-1.184117e+08,5.704000e+03,1.0,420.000000,0.000000,1.000000,1.000000,180.000000,100.000000,1.000000,8.127175e+04,1.990560e+05,8.228500e+04,2873.260000,4.000000,2016.0
50%,616.000000,2.000000,3.000000,2.000000,1244.000000,1540.000000,1518.000000,1440.000000,2104.000000,1248.000000,1986.000000,1.000000,2.000000,2.000000,433.000000,3.402153e+07,-1.181734e+08,7.200000e+03,1.0,500.000000,0.000000,1.000000,1.000000,260.000000,159.000000,1.000000,1.320570e+05,3.429310e+05,1.930000e+05,4543.100000,6.000000,2016.0
75%,872.000000,3.000000,4.000000,3.000000,1614.000000,2095.000000,2056.000000,1440.000000,2863.000000,1617.000000,3405.500000,1.000000,3.000000,2.000000,484.000000,3.417278e+07,-1.179216e+08,1.168175e+04,1.0,600.000000,0.000000,1.000000,1.000000,384.000000,361.000000,2.000000,2.105380e+05,5.405890e+05,3.453840e+05,6900.165000,8.000000,2016.0
max,1555.000000,20.000000,16.000000,20.000000,7625.000000,22741.000000,20013.000000,1584.000000,22741.000000,8352.000000,7224.000000,5.000000,20.000000,24.000000,7339.000000,3.481601e+07,-1.175549e+08,6.971010e+06,1.0,1750.000000,18.000000,4.000000,143.000000,2678.000000,1366.000000,4.000000,9.948100e+06,2.775000e+07,2.450000e+07,321936.090000,12.000000,2016.0


As we can see above in the minimum row, there are no records with negative values. Hence lets impute the missing data with -1

In [37]:
class DataProcessor:
    def __init__(self, cols_to_remove=None, datecol=None):
        self.cols_to_remove = cols_to_remove
        self.datecol = datecol

    def fit(self, X, y=None):
        """fit the process on the training data"""

        return self

    def transform(self, X, y=None):
        """transform the process on the train/test data """

        X_new = X.drop(columns=self.cols_to_remove, axis=1)

        if self.datecol:
            X_new[self.datecol + "_month"] = pd.to_datetime(
                X_new[self.datecol]
            ).dt.month
            X_new[self.datecol + "_year"] = pd.to_datetime(X_new[self.datecol]).dt.year
            X_new = X_new.drop(columns=self.datecol, axis=1)

        X_new = X_new.fillna(-1)

        return X_new

    def fit_transform(self, X, y=None):
        """fit and transform"""

        return self.fit(X).transform(X)

In [38]:
dp = DataProcessor(
    cols_to_remove=[
        "parcelid",
        "propertyzoningdesc",
        "rawcensustractandblock",
        "regionidneighborhood",
        "regionidzip",
        "censustractandblock",
    ],
    datecol="transactiondate",
)
X_train_new = dp.transform(X_train)

In [39]:
X_train_new.info()  # no null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 12731
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   airconditioningtypeid         90150 non-null  float64
 1   architecturalstyletypeid      90150 non-null  float64
 2   basementsqft                  90150 non-null  float64
 3   bathroomcnt                   90150 non-null  float64
 4   bedroomcnt                    90150 non-null  float64
 5   buildingclasstypeid           90150 non-null  float64
 6   buildingqualitytypeid         90150 non-null  float64
 7   calculatedbathnbr             90150 non-null  float64
 8   decktypeid                    90150 non-null  float64
 9   finishedfloor1squarefeet      90150 non-null  float64
 10  calculatedfinishedsquarefeet  90150 non-null  float64
 11  finishedsquarefeet12          90150 non-null  float64
 12  finishedsquarefeet13          90150 non-null  float64
 13  f

## One-Hot Encoding

We have a lot of categorical variables that needs to be encoded as real numbers which will help with modeling. One-hot encoding is better than label encoding as label encoding will not work with regression type models.

Before One-hot encoding, lets also classify the columns as numerical or categorical manually

In [40]:
class DataProcessor:
    def __init__(self, cols_to_remove=None, datecol=None):
        self.cols_to_remove = cols_to_remove
        self.datecol = datecol
        self.was_fit = False

    def fit(self, X, y=None):
        """fit the process on the training data"""

        self.was_fit = True

        # remove the columns
        X_new = X.drop(columns=self.cols_to_remove, axis=1)

        # get the categorical features
        self.categorical_features = X_new.dtypes[X_new.dtypes == "object"].index

        # dummy encoding
        dummy_df = pd.get_dummies(
            X_new, columns=self.categorical_features, dummy_na=True
        )
        self.allcols = dummy_df.columns

        return self

    def transform(self, X, y=None):
        """transform the process on the train/test data """

        if not self.was_fit:
            raise Error("Fit the DataProcessor first")

        # remove the columns
        X_new = X.drop(columns=self.cols_to_remove, axis=1)

        # get the categorical features
        self.categorical_features = X_new.dtypes[X_new.dtypes == "object"].index

        # dummy encoding
        X_new = pd.get_dummies(X_new, columns=self.categorical_features, dummy_na=True)

        # this is for test - make sure the dummy columns that are not in test but present in train are set to 0
        newcols = set(self.allcols) - set(X_new.columns)
        if newcols:
            for col in newcols:
                X_new[col] = 0

        X_new = X_new[self.allcols]

        # Create month and year columns for the transactiondate and drop transactiondate
        if self.datecol:
            X_new[self.datecol + "_month"] = pd.to_datetime(
                X_new[self.datecol]
            ).dt.month
            X_new[self.datecol + "_year"] = pd.to_datetime(X_new[self.datecol]).dt.year
            X_new = X_new.drop(columns=self.datecol, axis=1)

        # fill NaN with -1
        X_new = X_new.fillna(-1)

        return X_new

    def fit_transform(self, X, y=None):
        """fit and transform"""

        return self.fit(X).transform(X)

In [41]:
dp = DataProcessor(
    cols_to_remove=[
        "parcelid",
        "propertyzoningdesc",
        "rawcensustractandblock",
        "regionidneighborhood",
        "regionidzip",
        "censustractandblock",
    ],
    datecol="transactiondate",
)
dp.fit(X_train)

In [42]:
X_train_transformed = dp.transform(X_train)

In [43]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90150 entries, 0 to 12731
Columns: 514 entries, basementsqft to transactiondate_year
dtypes: float64(30), int64(2), uint8(482)
memory usage: 64.1 MB


In [44]:
# Get the test data set and transform using dp object
X_test, y_test = get_data(data_string="test")

In [45]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77613 entries, 0 to 77612
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   parcelid                      77613 non-null  object        
 1   transactiondate               77613 non-null  datetime64[ns]
 2   airconditioningtypeid         25007 non-null  object        
 3   architecturalstyletypeid      207 non-null    object        
 4   basementsqft                  50 non-null     float64       
 5   bathroomcnt                   77579 non-null  float64       
 6   bedroomcnt                    77579 non-null  float64       
 7   buildingclasstypeid           15 non-null     object        
 8   buildingqualitytypeid         49809 non-null  object        
 9   calculatedbathnbr             76963 non-null  float64       
 10  decktypeid                    614 non-null    object        
 11  finishedfloor1squarefeet    

In [46]:
len(y_test)

77613

In [47]:
X_test_transformed = dp.transform(X_test)

In [48]:
X_test_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77613 entries, 0 to 77612
Columns: 514 entries, basementsqft to transactiondate_year
dtypes: float64(30), int64(28), uint8(456)
memory usage: 68.7 MB


In [49]:
assert all(X_train_transformed.isna().sum() == 0), "NaN values present"
# No NaN values present after the transformation

In [50]:
assert all(X_test_transformed.isna().sum() == 0), "NaN values present"
# No NaN values present after the transformation

In [51]:
assert set(X_train_transformed.columns) == set(
    X_test_transformed.columns
), "both don't match"
# columns are same in train and test